# Proyek Analisis Data: E-Commerce Public
- **Nama:** Rafael Simson Riston
- **Email:** rafaelsimsonriston@gmail.com
- **ID Dicoding:** rafaelsimsonr

## About Dataset

Brazilian E-Commerce Public Dataset by Olist

Welcome! This is a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. We also released a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.

This is real commercial data, it has been anonymised, and references to the companies and partners in the review text have been replaced with the names of Game of Thrones great houses.

Context
This dataset was generously provided by Olist, the largest department store in Brazilian marketplaces. Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners. See more on our website: www.olist.com

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

Attention
1. An order might have multiple items.
2. Each item might be fulfilled by a distinct seller.
3. All text identifying stores and partners where replaced by the names of Game of Thrones great houses.

## Menentukan Pertanyaan

1. What is the percentage of product sales that weigh more than 1 kg in the last six months?
2. What is the average review score for each product category in the last three months?
3. What is the most effective pricing strategy to increase sales of products with an average review score below 3?
4. How does the payment method affect the number of transactions?
5. What are the sales trends for products by category over the last two years?
6. What is the geographical distribution of product sales based on the postal codes of sellers and buyers in the last year?


## 1. Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

## 2. Data Wrangling

### 1. Gathering Data

In [2]:
customers_df = pd.read_csv(os.path.join(os.getcwd(), 'data/customers_dataset.csv'))
geo_df = pd.read_csv(os.path.join(os.getcwd(), 'data/geolocation_dataset.csv'))
order_items_df = pd.read_csv(os.path.join(os.getcwd(), 'data/order_items_dataset.csv'))
order_pay_df = pd.read_csv(os.path.join(os.getcwd(), 'data/order_payments_dataset.csv'))
order_reviews_df = pd.read_csv(os.path.join(os.getcwd(), 'data/order_reviews_dataset.csv'))
orders_df = pd.read_csv(os.path.join(os.getcwd(), 'data/orders_dataset.csv'))
product_category_name_df = pd.read_csv(os.path.join(os.getcwd(), 'data/product_category_name_translation.csv'))
products_df = pd.read_csv(os.path.join(os.getcwd(), 'data/products_dataset.csv'))
sellers_df = pd.read_csv(os.path.join(os.getcwd(), 'data/sellers_dataset.csv'))

### 2. Accesing Data

#### a. Find missing values and duplicate values

In [3]:
# Assign df_names and dataframes
dataframe_names = ['sellers_df', 'products_df', 'product_category_name_df', 
                'orders_df', 'order_reviews_df', 'order_pay_df', 
                'order_items_df', 'geo_df', 'customers_df']
dataframes = [sellers_df, products_df, product_category_name_df, orders_df, 
              order_reviews_df, order_pay_df, order_items_df, geo_df, customers_df]

In [4]:
def missing_values_check(dataframe_names, dataframes):

    # Create a dictionary to store check results
    datas_check = {
        'data_name': [], 
        'n_rows': [], 
        'n_cols': [], 
        'sum_null': [], 
        'sum_col_null':[], 
        'name_col_null':[], 
        'sum_duplicated':[], 
        'sum_col_duplicate':[],
        'name_col_duplicate':[]
    }

    # Loop through dataframes and perform checks
    for data_name, data in zip(dataframe_names, dataframes):
        datas_check['data_name'].append(data_name)
        datas_check['n_rows'].append(data.shape[0])
        datas_check['n_cols'].append(data.shape[1])
        datas_check['sum_null'].append(data.isna().sum().sum())
        datas_check['sum_duplicated'].append(data.duplicated().sum().sum())
        
        # Initialize lists for storing column-wise null and duplicate information
        sum_col_null = []
        name_col_null = []
        sum_col_duplicate = []
        name_col_duplicate = []
        
        # Loop through columns of each dataframe
        for col in data.columns:
            # Count null values and duplicates for each column
            sum_col_null.append(data[col].isna().sum())
            sum_col_duplicate.append(data.duplicated().sum().sum())
        
        # Count columns with null and duplicate values
        sum_col_n = sum(n != 0 for n in sum_col_null)
        sum_col_d = sum(n != 0 for n in sum_col_duplicate)
        
        # Append column names with null and duplicate values
        for idx, (n_null, n_duplicate) in enumerate(zip(sum_col_null, sum_col_duplicate)):
            if n_null != 0:
                name_col_null.append(data.columns[idx])
            if n_duplicate != 0:
                name_col_duplicate.append(data.columns[idx])

        datas_check['sum_col_null'].append(sum_col_n)
        datas_check['name_col_null'].append(name_col_null)
        datas_check['sum_col_duplicate'].append(sum_col_d)
        datas_check['name_col_duplicate'].append(name_col_duplicate)

    # Convert dictionary to dataframe
    return pd.DataFrame(datas_check)


In [5]:
missing_values_check(dataframe_names, dataframes)

,data_name,n_rows,n_cols,sum_null,sum_col_null,name_col_null,sum_duplicated,sum_col_duplicate,name_col_duplicate
0,sellers_df,3095,4,0,0,[],0,0,[]
1,products_df,32951,9,2448,8,"[product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm]",0,0,[]
2,product_category_name_df,71,2,0,0,[],0,0,[]
3,orders_df,99441,8,4908,3,"[order_approved_at, order_delivered_carrier_date, order_delivered_customer_date]",0,0,[]
4,order_reviews_df,99224,7,145903,2,"[review_comment_title, review_comment_message]",0,0,[]
5,order_pay_df,103886,5,0,0,[],0,0,[]
6,order_items_df,112650,7,0,0,[],0,0,[]
7,geo_df,1000163,5,0,0,[],261831,5,"[geolocation_zip_code_prefix, geolocation_lat, geolocation_lng, geolocation_city, geolocation_state]"
8,customers_df,99441,5,0,0,[],0,0,[]


As you can see, there are a lot of missing values in the products dataset, orders, and reviews. On the other hand, duplicate values can be found in this data, specifically in the geographical dataset. However, this is normal because of the zip code, city, and state.

#### b. Check Data Types and Descriptive Statistics

In [6]:
# Initialize dict for store data
def describe_data(dataframe_names, dataframes):
    datas_describe = {
        'data_name':[],
        'column_name': [],
        'n_null':[],
        'n_null_%':[],
        'n_duplicated':[],
        'dtype': [],
        'count':[],
        'mean':[],
        'median':[],
        'min':[],
        'max':[]

    }

    # Use for loop to iterate to all dataframe
    for data_name, data in zip(dataframe_names, dataframes):
        for col in data.columns:
            n_null = data[col].isna().sum() 

            datas_describe['data_name'].append(data_name)
            datas_describe['column_name'].append(col)
            datas_describe['dtype'].append(str(data[col].dtype))  # Convert dtype to string
            datas_describe['count'].append(data[col].count())  # Use count() for non-null values count
            datas_describe['mean'].append(f"{data[col].mean():.2f}" if data[col].dtype in ['int64','float64'] else "")  # Calculate mean for numeric columns
            datas_describe['min'].append(f"{data[col].min():.2f}" if data[col].dtype in ['int64','float64'] else "")  # Calculate min for numeric columns
            datas_describe['max'].append(f"{data[col].max():.2f}" if data[col].dtype in ['int64','float64'] else "")  # Calculate max for numeric columns
            datas_describe['median'].append(f"{data[col].median():.2f}" if data[col].dtype == ['int','float'] else "")  # Calculate median for numeric columns
            datas_describe['n_null'].append(n_null) # Calculate null 
            datas_describe['n_null_%'].append(f'{round(n_null/len(data)*100)}%') # Calculate percentage null
            datas_describe['n_duplicated'].append(data[col].duplicated().sum())

    # For the purpose of readability, using pandas dataframe is the solution
    return pd.DataFrame(datas_describe)


In [7]:
describe_data(dataframe_names,dataframes)

,data_name,column_name,n_null,n_null_%,n_duplicated,dtype,count,mean,median,min,max
0,sellers_df,seller_id,0,0%,0,object,3095,,,,
1,sellers_df,seller_zip_code_prefix,0,0%,849,int64,3095,32291.06,,1001.00,99730.00
2,sellers_df,seller_city,0,0%,2484,object,3095,,,,
3,sellers_df,seller_state,0,0%,3072,object,3095,,,,
4,products_df,product_id,0,0%,0,object,32951,,,,
5,products_df,product_category_name,610,2%,32877,object,32341,,,,
6,products_df,product_name_lenght,610,2%,32884,float64,32341,48.48,,5.00,76.00
7,products_df,product_description_lenght,610,2%,29990,float64,32341,771.50,,4.00,3992.00
8,products_df,product_photos_qty,610,2%,32931,float64,32341,2.19,,1.00,20.00
9,products_df,product_weight_g,2,0%,30746,float64,32949,2276.47,,0.00,40425.00


There are many instances of incorrect data types and missing values in the order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, review_comment_title, review_comment_message, product_name_length, product_description_length, product_photos_qty, and product_category_name fields.

Most of the missing values are in string fields, with only a few in integer fields.

|         Column                | Data Type                 |           
| ----------------------------- | ------------------------- |
| order_purchase_timestamp      | **str** -> **datetime**   |
| order_approved_at             | **str** -> **datetime**   |
| order_delivered_carrier_date  | **str** -> **datetime**   |
| order_delivered_customer_date | **str** -> **datetime**   |
| order_estimated_delivery_date | **str** -> **datetime**   |
| review_creation_date          | **str** -> **datetime**   |
| review_answer_timestamp       | **str** -> **datetime**   |
| shipping_limit_date           | **str** -> **datetime**   |
| order_item_id                 | **int** -> **str**        |
| geolocation_zip_code_prefix   | **int** -> **str**        |
| customer_zip_code_prefix      | **int** -> **str**        |
| seller_zip_code_prefix        | **int** -> **str**        |


Also, check the payment_value column in the order payments data, because there are transactions with a payment of 0. There are zero values in the product_weight_g column in products_df, and in the freight_value column in order_items_df.

In [8]:
order_pay_df[order_pay_df['payment_value'] == order_pay_df['payment_value'].min()]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
19922,8bcbe01d44d147f901cd3192671144db,4,voucher,1,0.0
36822,fa65dad1b0e818e3ccc5cb0e39231352,14,voucher,1,0.0
43744,6ccb433e00daae1283ccc956189c82ae,4,voucher,1,0.0
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.0
62674,45ed6e85398a87c253db47c2d9f48216,3,voucher,1,0.0
77885,fa65dad1b0e818e3ccc5cb0e39231352,13,voucher,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.0
100766,b23878b3e8eb4d25a158f57d96331b18,4,voucher,1,0.0


Most orders have a payment of 0 because of vouchers, but some have a payment_type that is not defined. So, try changing it to ‘voucher’.

In [9]:
# Check product_weight_g column
products_df[products_df['product_weight_g'] == products_df['product_weight_g'].min()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
9769,81781c0fed9fe1ad6e8c81fca1e1cb08,cama_mesa_banho,51.0,529.0,1.0,0.0,30.0,25.0,30.0
13683,8038040ee2a71048d4bdbbdc985b69ab,cama_mesa_banho,48.0,528.0,1.0,0.0,30.0,25.0,30.0
14997,36ba42dd187055e1fbe943b2d11430ca,cama_mesa_banho,53.0,528.0,1.0,0.0,30.0,25.0,30.0
32079,e673e90efa65a5409ff4196c038bb5af,cama_mesa_banho,53.0,528.0,1.0,0.0,30.0,25.0,30.0


In [10]:
filter_volume = ((products_df['product_length_cm'] == 30) & (products_df['product_height_cm'] == 25) & (products_df['product_width_cm'] == 30))
filter_product_volume = (products_df['product_category_name'] == 'cama_mesa_banho') & filter_volume
products_df[filter_product_volume]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1731,500909059ad72b36b2554150cc327edb,cama_mesa_banho,39.0,1140.0,1.0,3100.0,30.0,25.0,30.0
9769,81781c0fed9fe1ad6e8c81fca1e1cb08,cama_mesa_banho,51.0,529.0,1.0,0.0,30.0,25.0,30.0
13683,8038040ee2a71048d4bdbbdc985b69ab,cama_mesa_banho,48.0,528.0,1.0,0.0,30.0,25.0,30.0
14997,36ba42dd187055e1fbe943b2d11430ca,cama_mesa_banho,53.0,528.0,1.0,0.0,30.0,25.0,30.0
32079,e673e90efa65a5409ff4196c038bb5af,cama_mesa_banho,53.0,528.0,1.0,0.0,30.0,25.0,30.0


For product_weight_g column that has zero, we'll change the value into 3100 g

In [11]:
# Check freight_value 
order_items_df[order_items_df['freight_value'] == order_items_df['freight_value'].min()]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
114,00404fa7a687c8c44ca69d42695aae73,1,53b36df67ebb7c41585e8d54d6772e08,7d13fca15225358621be4086e1eb0964,2018-05-15 04:31:26,99.9,0.0
258,00a870c6c06346e85335524935c600c0,1,aca2eb7d00ea1a7b8ebd4e68314663af,955fee9216a65b617aa5c0531780ce60,2018-05-14 00:14:29,69.9,0.0
483,011c899816ea29773525bd3322dbb6aa,1,53b36df67ebb7c41585e8d54d6772e08,7d13fca15225358621be4086e1eb0964,2018-05-07 05:30:45,99.9,0.0
508,012b3f6ab7776a8ab3443a4ad7bef2e6,1,422879e10f46682990de24d770e7f83d,1f50f920176fa81dab994f9023523100,2018-05-09 21:30:50,53.9,0.0
509,012b3f6ab7776a8ab3443a4ad7bef2e6,2,422879e10f46682990de24d770e7f83d,1f50f920176fa81dab994f9023523100,2018-05-09 21:30:50,53.9,0.0
...,...,...,...,...,...,...,...
111094,fc698f330ec7fb74859071cc6cb29772,1,422879e10f46682990de24d770e7f83d,1f50f920176fa81dab994f9023523100,2018-04-25 02:31:57,53.9,0.0
111497,fd4907109f6bac23f07064af84bec02d,1,7a10781637204d8d10485c71a6108a2e,4869f7a5dfa277a7dca6462dcf3b52b2,2018-04-30 11:31:32,219.0,0.0
111649,fd95e4b85ebbb81853d4a6be3d61432b,1,53b36df67ebb7c41585e8d54d6772e08,4869f7a5dfa277a7dca6462dcf3b52b2,2018-05-04 11:10:31,106.9,0.0
112182,fee19a0dc7358b6962a611cecf6a37b4,1,f1c7f353075ce59d8a6f3cf58f419c9c,37be5a7c751166fbc5f8ccba4119e043,2017-09-07 22:06:31,195.0,0.0


In [12]:
order_items_df[order_items_df['freight_value'] != order_items_df['freight_value'].min()]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [13]:
order_items_df[order_items_df['freight_value'] == order_items_df['freight_value'].min()].shape[0] / order_items_df['freight_value'].shape[0] *100

0.33999112294718153

We could drop the rows with a freight_value of zero, because it wouldn’t ruined the information in our data. But for now just keep it

Next, check the product_category_name in products_df, and compare it to product_category_name_df

In [14]:
len(products_df['product_category_name'].unique())

74

In [15]:
len(product_category_name_df['product_category_name'])

71

There're three products different from product_category_name_df

In [16]:
# Filter product_df 
# if product name in products_df != product name in product_category_name_df return True (~) 
mask = ~products_df['product_category_name'].isin(product_category_name_df['product_category_name']) 
filtered_products_name_df = products_df[mask]
filtered_products_name_df['product_category_name'].unique()

array([nan, 'pc_gamer', 'portateis_cozinha_e_preparadores_de_alimentos'],
      dtype=object)

In [17]:
filtered_products_name_df.shape[0] / products_df.shape[0] * 100

1.890686170374192

We could drop it, as it would have no impact on our datasets, given that we would only lose 2% of the data.

### 3. Cleaning Data

Fix the incorrect data types

|         Column                | Data Type                 |  Dataset          |
| ----------------------------- | ------------------------- | ------------------|
| order_purchase_timestamp      | **str** -> **datetime**   | orders_df         |
| order_approved_at             | **str** -> **datetime**   | orders_df         |
| order_delivered_carrier_date  | **str** -> **datetime**   | orders_df         |
| order_delivered_customer_date | **str** -> **datetime**   | orders_df         |
| order_estimated_delivery_date | **str** -> **datetime**   | orders_df         |
| review_creation_date          | **str** -> **datetime**   | order_reviews_df  |
| review_answer_timestamp       | **str** -> **datetime**   | order_reviews_df  |
| shipping_limit_date           | **str** -> **datetime**   | order_items_df    |
| order_item_id                 | **int** -> **str**        | order_items_df    |
| geolocation_zip_code_prefix   | **int** -> **str**        | geo_df	        |
| customer_zip_code_prefix      | **int** -> **str**        | customers_df      |
| seller_zip_code_prefix        | **int** -> **str**        | sellers_df        |

In [18]:
# Create a dictionary that maps column names to their new data types
column_dtype_mapping = {
    'orders_df': {
        'order_purchase_timestamp': 'datetime64[ns]',
        'order_approved_at': 'datetime64[ns]',
        'order_delivered_carrier_date': 'datetime64[ns]',
        'order_delivered_customer_date': 'datetime64[ns]',
        'order_estimated_delivery_date': 'datetime64[ns]'
    },
    'order_reviews_df': {
        'review_creation_date': 'datetime64[ns]',
        'review_answer_timestamp': 'datetime64[ns]'
    },
    'order_items_df': {
        'shipping_limit_date': 'datetime64[ns]',
        'order_item_id': 'str'
    },
    'geo_df': {
        'geolocation_zip_code_prefix': 'str'
    },
    'customers_df': {
        'customer_zip_code_prefix': 'str'
    },
    'sellers_df': {
        'seller_zip_code_prefix': 'str'
    }
}

# Use a loop to apply the changes
for df_name, columns in column_dtype_mapping.items():
    for column, dtype in columns.items():
        globals()[df_name][column] = globals()[df_name][column].astype(dtype)


In [19]:
describe_data(dataframe_names, dataframes)

,data_name,column_name,n_null,n_null_%,n_duplicated,dtype,count,mean,median,min,max
0,sellers_df,seller_id,0,0%,0,object,3095,,,,
1,sellers_df,seller_zip_code_prefix,0,0%,849,object,3095,,,,
2,sellers_df,seller_city,0,0%,2484,object,3095,,,,
3,sellers_df,seller_state,0,0%,3072,object,3095,,,,
4,products_df,product_id,0,0%,0,object,32951,,,,
5,products_df,product_category_name,610,2%,32877,object,32341,,,,
6,products_df,product_name_lenght,610,2%,32884,float64,32341,48.48,,5.00,76.00
7,products_df,product_description_lenght,610,2%,29990,float64,32341,771.50,,4.00,3992.00
8,products_df,product_photos_qty,610,2%,32931,float64,32341,2.19,,1.00,20.00
9,products_df,product_weight_g,2,0%,30746,float64,32949,2276.47,,0.00,40425.00


In [20]:
# Filter order_pay_df that contain payment_type not_defined and payment_value of zero.
order_pay_not_defined = (order_pay_df['payment_type'] == 'not_defined') & (order_pay_df['payment_value'] == order_pay_df['payment_value'].min())

In [21]:
# Replace not_defined to voucher
order_pay_df.loc[order_pay_not_defined, :] = order_pay_df.loc[order_pay_not_defined, :].replace('not_defined', 'voucher')

In [22]:
order_pay_df[order_pay_df['payment_value'] == order_pay_df['payment_value'].min()]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
19922,8bcbe01d44d147f901cd3192671144db,4,voucher,1,0.0
36822,fa65dad1b0e818e3ccc5cb0e39231352,14,voucher,1,0.0
43744,6ccb433e00daae1283ccc956189c82ae,4,voucher,1,0.0
51280,4637ca194b6387e2d538dc89b124b0ee,1,voucher,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,voucher,1,0.0
62674,45ed6e85398a87c253db47c2d9f48216,3,voucher,1,0.0
77885,fa65dad1b0e818e3ccc5cb0e39231352,13,voucher,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,voucher,1,0.0
100766,b23878b3e8eb4d25a158f57d96331b18,4,voucher,1,0.0


We fixed the payment_type

In [23]:
# Remove missing value in products_df
products_df.dropna(axis=0, inplace=True)

In [24]:
products_df[mask.reindex(products_df.index)]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1628,0105b5323d24fc655f73052694dbbb3a,pc_gamer,59.0,621.0,4.0,2839.0,19.0,16.0,18.0
5821,6fd83eb3e0799b775e4f946bd66657c0,portateis_cozinha_e_preparadores_de_alimentos,52.0,280.0,1.0,1200.0,25.0,33.0,25.0
7325,5d923ead886c44b86845f69e50520c3e,portateis_cozinha_e_preparadores_de_alimentos,58.0,284.0,1.0,1200.0,25.0,33.0,25.0
7478,6727051471a0fc4a0e7737b57bff2549,pc_gamer,60.0,1532.0,3.0,650.0,16.0,22.0,20.0
8819,bed164d9d628cf0593003389c535c6e0,portateis_cozinha_e_preparadores_de_alimentos,54.0,382.0,2.0,850.0,30.0,21.0,22.0
11039,1220978a08a6b29a202bc015b18250e9,portateis_cozinha_e_preparadores_de_alimentos,46.0,280.0,1.0,1200.0,25.0,33.0,25.0
14266,ae62bb0f95af63d64eae5f93dddea8d3,portateis_cozinha_e_preparadores_de_alimentos,59.0,927.0,1.0,10600.0,40.0,20.0,38.0
16182,1954739d84629e7323a4295812a3e0ec,portateis_cozinha_e_preparadores_de_alimentos,58.0,792.0,4.0,750.0,30.0,30.0,30.0
16930,dbe520fb381ad695a7e1f2807d20c765,pc_gamer,60.0,840.0,6.0,800.0,18.0,22.0,22.0
17800,c7a3f1a7f9eef146cc499368b578b884,portateis_cozinha_e_preparadores_de_alimentos,52.0,1372.0,5.0,7350.0,40.0,30.0,23.0


In [25]:
# Remove product_category_name
products_df = products_df[~mask.reindex(products_df.index)]

We fixed the missing product_category_name

In [26]:
products_df[filter_product_volume]

C:\Users\rafae\AppData\Local\Temp\ipykernel_18344\4041168729.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  products_df[filter_product_volume]


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1731,500909059ad72b36b2554150cc327edb,cama_mesa_banho,39.0,1140.0,1.0,3100.0,30.0,25.0,30.0
9769,81781c0fed9fe1ad6e8c81fca1e1cb08,cama_mesa_banho,51.0,529.0,1.0,0.0,30.0,25.0,30.0
13683,8038040ee2a71048d4bdbbdc985b69ab,cama_mesa_banho,48.0,528.0,1.0,0.0,30.0,25.0,30.0
14997,36ba42dd187055e1fbe943b2d11430ca,cama_mesa_banho,53.0,528.0,1.0,0.0,30.0,25.0,30.0
32079,e673e90efa65a5409ff4196c038bb5af,cama_mesa_banho,53.0,528.0,1.0,0.0,30.0,25.0,30.0


In [27]:
products_df.loc[filter_product_volume & (products_df['product_weight_g'] == 0), 'product_weight_g'] = 3100

We fixed the product_weight_g in products_df

Check the dimensions of geo_df and remove duplicated values in the zip code because the zip code is a unique identifier for geographical locations.

In [28]:
# Check the dimensions before removing duplicates
geo_df.shape

(1000163, 5)

In [29]:
# Check how much duplicated values in zip code
geo_df['geolocation_zip_code_prefix'].duplicated().sum()

981148

In [30]:
# Remove duplicates
geo_df = geo_df.drop_duplicates(subset='geolocation_zip_code_prefix', keep='last')
geo_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
950,1029,-23.539609,-46.632911,sao paulo,SP
962,1006,-23.550317,-46.636604,são paulo,SP
1127,1038,-23.544340,-46.640552,sao paulo,SP
1133,1018,-23.551846,-46.633756,sao paulo,SP
1156,1022,-23.547795,-46.630238,sao paulo,SP
...,...,...,...,...,...
1000153,99970,-28.343273,-51.873734,ciriaco,RS
1000155,99965,-28.180655,-52.034367,agua santa,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


## 3. Explonatory Data Analysis

In [31]:
describe_data(dataframe_names,dataframes)[['data_name','column_name']]

,data_name,column_name
0,sellers_df,seller_id
1,sellers_df,seller_zip_code_prefix
2,sellers_df,seller_city
3,sellers_df,seller_state
4,products_df,product_id
5,products_df,product_category_name
6,products_df,product_name_lenght
7,products_df,product_description_lenght
8,products_df,product_photos_qty
9,products_df,product_weight_g


Combine all dataframes and select only the features necessary for analysis.

In [32]:
all_df = pd.merge(orders_df, order_items_df, on='order_id')
all_df = pd.merge(all_df, sellers_df, on='seller_id')
all_df = pd.merge(all_df, order_pay_df, on='order_id')
all_df = pd.merge(all_df, order_reviews_df, on='order_id')
all_df = pd.merge(all_df, products_df, on='product_id')
all_df = pd.merge(all_df, customers_df, on='customer_id')
all_df = pd.merge(all_df, product_category_name_df, on='product_category_name')
all_df = pd.merge(all_df, geo_df, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
all_df = pd.merge(all_df, geo_df, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')
all_df.shape

(115036, 50)

Remove and rename the columns that are necessary.|

In [33]:
# Remove columns
all_df.drop(columns=['review_answer_timestamp', 'review_creation_date', 'review_comment_message','review_comment_title', 
                    'customer_zip_code_prefix', 'seller_zip_code_prefix','product_category_name','review_id', 'product_description_lenght', 
                     'product_name_lenght', 'product_photos_qty', 'product_name_lenght', 'payment_installments', 'geolocation_city_x',
                    'geolocation_state_x', 'geolocation_state_y', 'geolocation_city_y'], inplace=True)
# Rename Columns
all_df.rename(columns={
    'geolocation_zip_code_prefix_x':'zip_code_customer',
    'geolocation_zip_code_prefix_y':'zip_code_seller',
    'geolocation_lat_x':'customer_lat',
    'geolocation_lng_x':'customer_lng',
    'geolocation_lat_y':'seller_lat',
    'geolocation_lng_y':'seller_lng',
},inplace=True)

all_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,payment_sequential,payment_type,payment_value,review_score,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_city,customer_state,product_category_name_english,zip_code_customer,customer_lat,customer_lng,zip_code_seller,seller_lat,seller_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,1,credit_card,18.12,4,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,housewares,3149,-23.583452,-46.586284,9350,-23.675542,-46.442547
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,3,voucher,2.00,4,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,housewares,3149,-23.583452,-46.586284,9350,-23.675542,-46.442547
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,2,voucher,18.59,4,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,housewares,3149,-23.583452,-46.586284,9350,-23.675542,-46.442547
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-21 20:05:16,29.99,7.78,maua,SP,1,credit_card,37.77,4,500.0,19.0,8.0,13.0,3a51803cc0d012c3b5dc8b7528cb05f7,sao paulo,SP,housewares,3366,-23.564977,-46.536043,9350,-23.675542,-46.442547
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-08 18:37:31,29.99,7.78,maua,SP,1,credit_card,37.77,5,500.0,19.0,8.0,13.0,ef0996a1a279c26e7ecbd737be23d235,sao paulo,SP,housewares,2290,-23.532175,-46.626106,9350,-23.675542,-46.442547


In [34]:
all_df.groupby(by='seller_city').agg({
    'order_id': 'nunique',
    'payment_value': ['mean','max','min']
}).sort_values(by=('order_id', 'nunique'),ascending=False).head()

order_id payment_value                
                nunique          mean       max   min
seller_city                                          
sao paulo         24026    144.036762  13664.08  0.01
ibitinga           6455    128.751346   1861.26  0.11
santo andre        2682    112.023474   1978.18  0.00
curitiba           2669    208.263415   3076.13  1.28
belo horizonte     2318    112.181051   3736.22  0.01

In [35]:
all_df.groupby(by='customer_city').agg({
    'order_id': 'nunique',
    'payment_value': ['mean','max','min']
}).sort_values(by=('order_id', 'nunique'),ascending=False).head()

order_id payment_value                
                nunique          mean       max   min
customer_city                                        
sao paulo         15010    153.803169   4513.32  0.01
rio de janeiro     6619    191.701207  13664.08  0.00
belo horizonte     2675    153.637031   3351.35  0.01
brasilia           1908    173.748090   3736.22  0.67
curitiba           1482    183.335207   3126.50  0.20

In [36]:
all_df.groupby(by='order_status').agg({
    'order_id': 'nunique',
    'payment_value': ['mean','max','min']
}).sort_values(by=('order_id', 'nunique'),ascending=False).head()

order_id payment_value                
              nunique          mean       max   min
order_status                                       
delivered       94018    171.607740  13664.08  0.00
shipped          1002    158.757120   2360.42  0.00
canceled          426    266.752377   4809.44  0.33
invoiced          298    223.472254   1827.72  1.02
processing        283    270.506685   2566.90  0.17

In [37]:
all_df.groupby(by='product_category_name_english').agg({
    'order_id': 'nunique',
    'payment_value': ['mean','max','min']
}).sort_values(by=('order_id', 'nunique'),ascending=False).head()

order_id payment_value               
                               nunique          mean      max   min
product_category_name_english                                      
bed_bath_table                    9295    145.550118  2225.69  0.01
health_beauty                     8741    165.728546  2938.17  0.03
sports_leisure                    7653    155.547542  4163.51  0.00
computers_accessories             6632    195.427430  3736.22  0.01
furniture_decor                   6381    163.317275  2276.10  0.01

In [38]:
all_df.groupby(by='payment_type').agg({
    'order_id': 'nunique',
    'payment_value': ['mean','max','min']
}).sort_values(by=('order_id', 'nunique'),ascending=False).head()

order_id payment_value                 
              nunique          mean       max    min
payment_type                                        
credit_card     73947    179.474720  13664.08   0.01
boleto          19111    176.856672   7274.88  11.62
voucher          3657     64.238536   1839.05   0.00
debit_card       1481    150.395390   4445.50  13.38

In [39]:
all_df['order_purchase_year'] = all_df['order_purchase_timestamp'].dt.year

In [40]:
all_df['order_purchase_month_name'] = all_df['order_purchase_timestamp'].dt.month_name()

In [41]:
all_df['order_purchase_day_name'] = all_df['order_purchase_timestamp'].dt.day_name()

In [42]:
def get_part_of_day(hour):
    if (hour >= 4) and (hour < 12):
        return 'morning'
    elif (hour >= 12) and (hour < 17):
        return 'afternoon'
    elif (hour >= 17) and (hour < 21):
        return 'evening'
    else:
        return 'night'

In [43]:
all_df['part_of_day'] = all_df['order_purchase_timestamp'].dt.hour.apply(get_part_of_day)

In [44]:
all_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,payment_sequential,payment_type,payment_value,review_score,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_city,customer_state,product_category_name_english,zip_code_customer,customer_lat,customer_lng,zip_code_seller,seller_lat,seller_lng,order_purchase_year,order_purchase_month_name,order_purchase_day_name,part_of_day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,1,credit_card,18.12,4,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,housewares,3149,-23.583452,-46.586284,9350,-23.675542,-46.442547,2017,October,Monday,morning
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,3,voucher,2.00,4,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,housewares,3149,-23.583452,-46.586284,9350,-23.675542,-46.442547,2017,October,Monday,morning
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,2,voucher,18.59,4,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,housewares,3149,-23.583452,-46.586284,9350,-23.675542,-46.442547,2017,October,Monday,morning
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-21 20:05:16,29.99,7.78,maua,SP,1,credit_card,37.77,4,500.0,19.0,8.0,13.0,3a51803cc0d012c3b5dc8b7528cb05f7,sao paulo,SP,housewares,3366,-23.564977,-46.536043,9350,-23.675542,-46.442547,2017,August,Tuesday,evening
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-08 18:37:31,29.99,7.78,maua,SP,1,credit_card,37.77,5,500.0,19.0,8.0,13.0,ef0996a1a279c26e7ecbd737be23d235,sao paulo,SP,housewares,2290,-23.532175,-46.626106,9350,-23.675542,-46.442547,2017,August,Wednesday,evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115031,a37e262a26000281d0ade3b4eb1ce1e6,d0602b514179e7ed50e801cc02769bac,delivered,2018-08-06 23:14:27,2018-08-06 23:25:19,2018-08-07 14:16:00,2018-08-13 19:08:40,2018-08-22,1,987ae6d6e94a608f9ea32bc82a649470,466222e777149751370e7e98fb623b0c,2018-08-08 23:25:19,87.90,17.55,rio de janeiro,RJ,1,credit_card,105.45,5,550.0,36.0,8.0,15.0,ce05c653348a8e630ebbb1fd1e0968d2,goiania,GO,auto,74080,-16.687639,-49.257401,20931,-22.892133,-43.218956,2018,August,Monday,night
115032,8edaa376e19d08bc84ab8845682216b4,4c1e29ec2ed2feac441cf24b25262ed2,delivered,2018-04-24 08:37:20,2018-04-24 17:26:25,2018-04-25 12:24:00,2018-05-10 22:36:37,2018-05-14,1,b0498e44190727b728ae4490f2e9b6a5,dda37071807e404c5bb2a1590c66326f,2018-04-30 09:30:37,199.99,19.28,sao paulo,SP,1,credit_card,219.27,5,5250.0,34.0,23.0,40.0,913a4e0cb7fe555e6cffa875ecd58e2e,sao jose de uba,RJ,luggage_accessories,28455,-21.363598,-41.939397,3282,-23.582798,-46.551769,201

## 4. Data Visualization & Explanatory Data Analysis

Pertanyaan Bisnis:
1. Berapa persentase penjualan produk yang beratnya lebih dari 1 kg dalam enam bulan terakhir?
2. Berapa rata-rata skor ulasan untuk setiap kategori produk dalam tiga bulan terakhir?
3. Apa strategi harga yang paling efektif untuk meningkatkan penjualan produk dengan skor ulasan rata-rata di bawah 3?
4. Bagaimana pengaruh antara metode pembayaran dan jumlah transaksi?
5. Bagaimana tren penjualan produk per kategori dalam dua tahun terakhir?
6. Bagaimana distribusi geografis penjualan produk berdasarkan kode pos penjual dan pembeli dalam satu tahun terakhir? 



## 5. Conclusion